In [5]:
%pip install pandas pytesseract pdf2image

  Using cached pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
Using cached pdf2image-1.17.0-py3-none-any.whl (11 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pytesseract
import pandas as pd
from pytesseract import Output
from pdf2image import convert_from_path  # Para converter PDFs em imagens
import os

In [9]:
# Defina o caminho do arquivo PDF
path_arquivo = '/Users/wrosalem/Documents/0x0-Coding/GitHub/rag-architecture-demo/Pipeline/joao_no_watermark.pdf'

# Verifica se o arquivo existe
if os.path.exists(path_arquivo):
    
    # Converte as páginas do PDF em uma lista de imagens
    try:
        paginas_imagens = convert_from_path(path_arquivo)
        
        for i, imagem_processada in enumerate(paginas_imagens):
            print(f"Processando página {i+1} do PDF...")
            
            # Configuração personalizada do Tesseract OCR
           # ocr_config = r'-c preserve_interword_spaces=1 --tessdata-dir tessdata --oem 1 --psm 1'  # Preserva espaços entre palavras
            ocr_config = r'-c preserve_interword_spaces=1 --tessdata-dir /opt/homebrew/share/tessdata --oem 1 --psm 1'


            # Executa o OCR
            ocr_data = pytesseract.image_to_data(imagem_processada, lang='por', config=ocr_config, output_type=Output.DICT)

            # Converte os dados em um DataFrame para análise
            ocr_dataframe = pd.DataFrame(ocr_data)

            # Limpeza de dados: remove espaços em branco e textos inválidos
            cleaned_df = ocr_dataframe[(ocr_dataframe.conf != '-1') & (ocr_dataframe.text != ' ') & (ocr_dataframe.text != '')]

            # Ordena os blocos de texto de forma vertical com base na posição 'top'
            sorted_block_numbers = cleaned_df.groupby('block_num').first().sort_values('top').index.tolist()

            for block_num in sorted_block_numbers:
                current_block = cleaned_df[cleaned_df['block_num'] == block_num]  # Filtra dados do bloco atual
                filtered_text = current_block[current_block.text.str.len() > 3]  # Seleciona textos com mais de 3 caracteres
                avg_char_width = (filtered_text.width / filtered_text.text.str.len()).mean()  # Calcula a largura média de um caractere
                prev_paragraph, prev_line, prev_left_margin = 0, 0, 0  # Variáveis para controlar parágrafos, linhas e margens anteriores
                extracted_text = ''  # Variável para armazenar o texto extraído

                for idx, line_data in current_block.iterrows():
                    # Muda de parágrafo ou linha conforme necessário
                    if prev_paragraph != line_data['par_num']:
                        extracted_text += '\n'
                        prev_paragraph = line_data['par_num']
                        prev_line = line_data['line_num']
                        prev_left_margin = 0
                    elif prev_line != line_data['line_num']:
                        extracted_text += '\n'
                        prev_line = line_data['line_num']
                        prev_left_margin = 0

                    # Calcula quantos espaços adicionar para alinhar o texto
                    spaces_to_add = 0
                    if line_data['left'] / avg_char_width > prev_left_margin + 1:
                        spaces_to_add = int((line_data['left']) / avg_char_width) - prev_left_margin
                        extracted_text += ' ' * spaces_to_add  # Adiciona os espaços necessários para manter o alinhamento
                    extracted_text += line_data['text'] + ' '  # Adiciona o texto da linha atual
                    prev_left_margin += len(line_data['text']) + spaces_to_add + 1  # Atualiza a margem esquerda para a próxima linha

                extracted_text += '\n'  # Adiciona uma nova linha ao final de cada bloco
                print(extracted_text)  # Exibe o texto extraído
                
    except Exception as e:
        print(f"Erro ao processar o PDF: {e}")
else:
    print(f"O arquivo {path_arquivo} não foi encontrado.")

Processando página 1 do PDF...

  NOME:  JOAO CARLOS MARCO 


   CPF: 816.283.600-46                               IMPOSTO SOBRE A RENDA - PESSOA FÍSICA 
   DECLARAÇÃO  DE AJUSTE ANUAL                       EXERCÍCIO 2023       ANO-CALENDÁRIO 2022 


   IDENTIFICAÇÃO DO CONTRIBUINTE 


    Nome:                            JOAO  CARLOS  MARCO                                        CPF:           816.283.600-46 
    Data de Nascimento:              09/10/1965                                                 Título Eleitoral: 263060660175 
    Possui cônjuge ou companheiro(a)? 
    Houve alteração de dados cadastrais? 


    Um dos declarantes é pessoa com doença grave ou portadora de deficiência física ou mental?       Não 
    Endereço:                    AVENIDA  DOS  ABACAXIS                                        Número:       100 
    Complemento:                                                                               Bairro/Distrito: CENTRO 
    Município:                   CA